In [ ]:
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from models_used import CIFAR10_Net
from torchvision import models
import json
from voting_utils import WeightFinding

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
# Define the directory containing the models
model_directory = 'cinic_various_models_2/'

# Function to load models with "fl" in their name
def load_models(directory, noise_type = 'fl'):
    models_loaded = []

    # List all files in the directory
    files = os.listdir(directory)
    
    for file in tqdm(files):
        if noise_type in file:
            # Load the model
            if 'cifar10_net' in file:
                model = CIFAR10_Net()
            else:
                model = models.vgg16(pretrained = True)
                input_lastLayer = model.classifier[6].in_features
                model.classifier[6] = nn.Linear(input_lastLayer,10)
            model_path = os.path.join(directory, file)
            model.load_state_dict(torch.load(model_path))
            models_loaded.append((file, model))
    
    return models_loaded

# Load the models
fl_models = load_models(model_directory)

# Example usage: Print the names of the loaded models
for file_name, model in fl_models:
    print(f"Loaded model from file: {file_name}")

In [ ]:
di_models = load_models(model_directory, noise_type = 'di')

# Example usage: Print the names of the loaded models
for file_name, model in di_models:
    print(f"Loaded model from file: {file_name}")
    # You can also print the model structure if needed
    # print(model)


### Load validation dataloaders

In [ ]:
cinic_directory = 'cinic_10_data'
cinic_mean = [0.47889522, 0.47227842, 0.43047404]
cinic_std = [0.24205776, 0.23828046, 0.25874835]

cifar_valid = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/valid2/cifar',
    	transform=transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=cinic_mean,std=cinic_std)])),
    batch_size=512, shuffle=False)

imagenet_valid = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder(cinic_directory + '/valid2/imagenet',
    	transform=transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=cinic_mean,std=cinic_std)])),
    batch_size=512, shuffle=False)

### Create weight finder objects

In [ ]:
fl_just_models = [model for file_name, model in fl_models]
di_just_models = [model for file_name, model in di_models]

fl_just_names = [file_name for file_name, model in fl_models]
di_just_names = [file_name for file_name, model in di_models]

In [ ]:
ci_loaders = [cifar_valid, imagenet_valid]
# c_loader = [cifar_valid]
# i_loader = [imagenet_valid]

#### Uncomment the cell below only if you're computing the weight finder objects for the first time

In [ ]:
wf_fl_ci = WeightFinding(fl_just_models, ci_loaders, device)
wf_di_ci = WeightFinding(di_just_models, ci_loaders, device)

# wf_fl_c = WeightFinding(fl_just_models, c_loader, device)
# wf_di_c = WeightFinding(di_just_models, c_loader, device)

# wf_fl_i = WeightFinding(fl_just_models, i_loader, device)
# wf_di_i = WeightFinding(di_just_models, i_loader, device)

# wf_fl_ci.models = []
# wf_di_ci.models = []
# wf_fl_c.models = []
# wf_di_c.models = []
# wf_fl_i.models = []
# wf_di_i.models = []

In [ ]:
# del fl_just_models
# del di_just_models

In [ ]:
# Save these objects
wf_fl_ci.save('weight_finders_2/wf_fl_ci.pkl')
wf_di_ci.save('weight_finders_2/wf_di_ci.pkl')
# wf_fl_c.save('weight_finders_2/wf_fl_c.pkl')
# wf_di_c.save('weight_finders_2/wf_di_c.pkl')
# wf_fl_i.save('weight_finders_2/wf_fl_i.pkl')
# wf_di_i.save('weight_finders_2/wf_di_i.pkl')

# Load the objects
wf_fl_ci = WeightFinding.load('weight_finders_2/wf_fl_ci.pkl')
wf_di_ci = WeightFinding.load('weight_finders_2/wf_di_ci.pkl')
# wf_fl_c = WeightFinding.load('weight_finders_2/wf_fl_c.pkl')
# wf_di_c = WeightFinding.load('weight_finders_2/wf_di_c.pkl')
# wf_fl_i = WeightFinding.load('weight_finders_2/wf_fl_i.pkl')
# wf_di_i = WeightFinding.load('weight_finders_2/wf_di_i.pkl')

### LOO Weights

In [ ]:
fl_loo_weights_borda_ci = wf_fl_ci.loo_pytorch(method = 'borda')
fl_loo_weights_borda_c = wf_fl_c.loo_pytorch(method = 'borda')
fl_loo_weights_borda_i = wf_fl_i.loo_pytorch(method = 'borda')

di_loo_weights_borda_ci = wf_di_ci.loo_pytorch(method = 'borda')
di_loo_weights_borda_c = wf_di_c.loo_pytorch(method = 'borda')
di_loo_weights_borda_i = wf_di_i.loo_pytorch(method = 'borda')

fl_loo_weights_plurality_ci = wf_fl_ci.loo_pytorch(method = 'plurality')
fl_loo_weights_plurality_c = wf_fl_c.loo_pytorch(method = 'plurality')
fl_loo_weights_plurality_i = wf_fl_i.loo_pytorch(method = 'plurality')

di_loo_weights_plurality_ci = wf_di_ci.loo_pytorch(method = 'plurality')
di_loo_weights_plurality_c = wf_di_c.loo_pytorch(method = 'plurality')
di_loo_weights_plurality_i = wf_di_i.loo_pytorch(method = 'plurality')

Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models
Calculating LOO values for 16 models


In [ ]:
# make dictionary of the weights with model names
fl_loo_weights_borda_ci_dict = dict(zip(fl_just_names, fl_loo_weights_borda_ci))
fl_loo_weights_borda_c_dict = dict(zip(fl_just_names, fl_loo_weights_borda_c))
fl_loo_weights_borda_i_dict = dict(zip(fl_just_names, fl_loo_weights_borda_i))
di_loo_weights_borda_ci_dict = dict(zip(di_just_names, di_loo_weights_borda_ci))
di_loo_weights_borda_c_dict = dict(zip(di_just_names, di_loo_weights_borda_c))
di_loo_weights_borda_i_dict = dict(zip(di_just_names, di_loo_weights_borda_i))

fl_loo_weights_plurality_ci_dict = dict(zip(fl_just_names, fl_loo_weights_plurality_ci))
fl_loo_weights_plurality_c_dict = dict(zip(fl_just_names, fl_loo_weights_plurality_c))
fl_loo_weights_plurality_i_dict = dict(zip(fl_just_names, fl_loo_weights_plurality_i))
di_loo_weights_plurality_ci_dict = dict(zip(di_just_names, di_loo_weights_plurality_ci))
di_loo_weights_plurality_c_dict = dict(zip(di_just_names, di_loo_weights_plurality_c))
di_loo_weights_plurality_i_dict = dict(zip(di_just_names, di_loo_weights_plurality_i))

In [ ]:
# Save the weights
with open('weights_2/loo/fl_loo_weights_borda_ci.json', 'w') as f:
    json.dump(fl_loo_weights_borda_ci_dict, f)

with open('weights_2/loo/fl_loo_weights_borda_c.json', 'w') as f:
    json.dump(fl_loo_weights_borda_c_dict, f)

with open('weights_2/loo/fl_loo_weights_borda_i.json', 'w') as f:
    json.dump(fl_loo_weights_borda_i_dict, f)

with open('weights_2/loo/di_loo_weights_borda_ci.json', 'w') as f:
    json.dump(di_loo_weights_borda_ci_dict, f)

with open('weights_2/loo/di_loo_weights_borda_c.json', 'w') as f:
    json.dump(di_loo_weights_borda_c_dict, f)

with open('weights_2/loo/di_loo_weights_borda_i.json', 'w') as f:
    json.dump(di_loo_weights_borda_i_dict, f)

with open('weights_2/loo/fl_loo_weights_plurality_ci.json', 'w') as f:
    json.dump(fl_loo_weights_plurality_ci_dict, f)

with open('weights_2/loo/fl_loo_weights_plurality_c.json', 'w') as f:
    json.dump(fl_loo_weights_plurality_c_dict, f)

with open('weights_2/loo/fl_loo_weights_plurality_i.json', 'w') as f:
    json.dump(fl_loo_weights_plurality_i_dict, f)

with open('weights_2/loo/di_loo_weights_plurality_ci.json', 'w') as f:
    json.dump(di_loo_weights_plurality_ci_dict, f)

with open('weights_2/loo/di_loo_weights_plurality_c.json', 'w') as f:
    json.dump(di_loo_weights_plurality_c_dict, f)

with open('weights_2/loo/di_loo_weights_plurality_i.json', 'w') as f:
    json.dump(di_loo_weights_plurality_i_dict, f)

# Load the weights
with open('weights_2/loo/fl_loo_weights_borda_ci.json', 'r') as f:
    fl_loo_weights_borda_ci_dict = json.load(f)

with open('weights_2/loo/fl_loo_weights_borda_c.json', 'r') as f:
    fl_loo_weights_borda_c_dict = json.load(f)

with open('weights_2/loo/fl_loo_weights_borda_i.json', 'r') as f:
    fl_loo_weights_borda_i_dict = json.load(f)

with open('weights_2/loo/di_loo_weights_borda_ci.json', 'r') as f:
    di_loo_weights_borda_ci_dict = json.load(f)

with open('weights_2/loo/di_loo_weights_borda_c.json', 'r') as f:
    di_loo_weights_borda_c_dict = json.load(f)

with open('weights_2/loo/di_loo_weights_borda_i.json', 'r') as f:
    di_loo_weights_borda_i_dict = json.load(f)

with open('weights_2/loo/fl_loo_weights_plurality_ci.json', 'r') as f:
    fl_loo_weights_plurality_ci_dict = json.load(f)

with open('weights_2/loo/fl_loo_weights_plurality_c.json', 'r') as f:
    fl_loo_weights_plurality_c_dict = json.load(f)

with open('weights_2/loo/fl_loo_weights_plurality_i.json', 'r') as f:
    fl_loo_weights_plurality_i_dict = json.load(f)

with open('weights_2/loo/di_loo_weights_plurality_ci.json', 'r') as f:
    di_loo_weights_plurality_ci_dict = json.load(f)

with open('weights_2/loo/di_loo_weights_plurality_c.json', 'r') as f:
    di_loo_weights_plurality_c_dict = json.load(f)

with open('weights_2/loo/di_loo_weights_plurality_i.json', 'r') as f:
    di_loo_weights_plurality_i_dict = json.load(f)

### CRH weights

In [ ]:
fl_crh_weights_ci = wf_fl_ci.crh_pytorch()
fl_crh_weights_c = wf_fl_c.crh_pytorch()
fl_crh_weights_i = wf_fl_i.crh_pytorch()

di_crh_weights_ci = wf_di_ci.crh_pytorch()
di_crh_weights_c = wf_di_c.crh_pytorch()
di_crh_weights_i = wf_di_i.crh_pytorch()

In [ ]:
fl_crh_weights_ci_dict = dict(zip(fl_just_names, fl_crh_weights_ci))
fl_crh_weights_c_dict = dict(zip(fl_just_names, fl_crh_weights_c))
fl_crh_weights_i_dict = dict(zip(fl_just_names, fl_crh_weights_i))
di_crh_weights_ci_dict = dict(zip(di_just_names, di_crh_weights_ci))
di_crh_weights_c_dict = dict(zip(di_just_names, di_crh_weights_c))
di_crh_weights_i_dict = dict(zip(di_just_names, di_crh_weights_i))

In [ ]:
# Save the CRH weights
with open('weights_2/crh/fl_crh_weights_ci.json', 'w') as f:
    json.dump(fl_crh_weights_ci_dict, f)

with open('weights_2/crh/fl_crh_weights_c.json', 'w') as f:
    json.dump(fl_crh_weights_c_dict, f)

with open('weights_2/crh/fl_crh_weights_i.json', 'w') as f:
    json.dump(fl_crh_weights_i_dict, f)

with open('weights_2/crh/di_crh_weights_ci.json', 'w') as f:
    json.dump(di_crh_weights_ci_dict, f)

with open('weights_2/crh/di_crh_weights_c.json', 'w') as f:
    json.dump(di_crh_weights_c_dict, f)

with open('weights_2/crh/di_crh_weights_i.json', 'w') as f:
    json.dump(di_crh_weights_i_dict, f)

# Load the CRH weights
with open('weights_2/crh/fl_crh_weights_ci.json', 'r') as f:
    fl_crh_weights_ci_dict = json.load(f)

with open('weights_2/crh/fl_crh_weights_c.json', 'r') as f:
    fl_crh_weights_c_dict = json.load(f)

with open('weights_2/crh/fl_crh_weights_i.json', 'r') as f:
    fl_crh_weights_i_dict = json.load(f)

with open('weights_2/crh/di_crh_weights_ci.json', 'r') as f:
    di_crh_weights_ci_dict = json.load(f)

with open('weights_2/crh/di_crh_weights_c.json', 'r') as f:
    di_crh_weights_c_dict = json.load(f)

with open('weights_2/crh/di_crh_weights_i.json', 'r') as f:
    di_crh_weights_i_dict = json.load(f)

### Entropy weights

In [ ]:
fl_entropy_weights_ci = wf_fl_ci.entropy_weights()
fl_entropy_weights_c = wf_fl_c.entropy_weights()
fl_entropy_weights_i = wf_fl_i.entropy_weights()

di_entropy_weights_ci = wf_di_ci.entropy_weights()
di_entropy_weights_c = wf_di_c.entropy_weights()
di_entropy_weights_i = wf_di_i.entropy_weights()

In [ ]:
fl_entropy_weights_ci_dict = dict(zip(fl_just_names, fl_entropy_weights_ci))
fl_entropy_weights_c_dict = dict(zip(fl_just_names, fl_entropy_weights_c))
fl_entropy_weights_i_dict = dict(zip(fl_just_names, fl_entropy_weights_i))
di_entropy_weights_ci_dict = dict(zip(di_just_names, di_entropy_weights_ci))
di_entropy_weights_c_dict = dict(zip(di_just_names, di_entropy_weights_c))
di_entropy_weights_i_dict = dict(zip(di_just_names, di_entropy_weights_i))

In [ ]:
# Save the Entropy weights
with open('weights_2/entropy/fl_entropy_weights_ci.json', 'w') as f:
    json.dump(fl_entropy_weights_ci_dict, f)

with open('weights_2/entropy/fl_entropy_weights_c.json', 'w') as f:
    json.dump(fl_entropy_weights_c_dict, f)

with open('weights_2/entropy/fl_entropy_weights_i.json', 'w') as f:
    json.dump(fl_entropy_weights_i_dict, f)

with open('weights_2/entropy/di_entropy_weights_ci.json', 'w') as f:
    json.dump(di_entropy_weights_ci_dict, f)

with open('weights_2/entropy/di_entropy_weights_c.json', 'w') as f:
    json.dump(di_entropy_weights_c_dict, f)

with open('weights_2/entropy/di_entropy_weights_i.json', 'w') as f:
    json.dump(di_entropy_weights_i_dict, f)

# Load the entropy weights
with open('weights_2/entropy/fl_entropy_weights_ci.json', 'r') as f:
    fl_entropy_weights_ci_dict = json.load(f)

with open('weights_2/entropy/fl_entropy_weights_c.json', 'r') as f:
    fl_entropy_weights_c_dict = json.load(f)

with open('weights_2/entropy/fl_entropy_weights_i.json', 'r') as f:
    fl_entropy_weights_i_dict = json.load(f)

with open('weights_2/entropy/di_entropy_weights_ci.json', 'r') as f:
    di_entropy_weights_ci_dict = json.load(f)

with open('weights_2/entropy/di_entropy_weights_c.json', 'r') as f:
    di_entropy_weights_c_dict = json.load(f)

with open('weights_2/entropy/di_entropy_weights_i.json', 'r') as f:
    di_entropy_weights_i_dict = json.load(f)

### Accuracy weights

In [ ]:
fl_accuracy_weights_ci = wf_fl_ci.accuracy_weights()
fl_accuracy_weights_c = wf_fl_c.accuracy_weights()
fl_accuracy_weights_i = wf_fl_i.accuracy_weights()

di_accuracy_weights_ci = wf_di_ci.accuracy_weights()
di_accuracy_weights_c = wf_di_c.accuracy_weights()
di_accuracy_weights_i = wf_di_i.accuracy_weights()

In [ ]:
fl_accuracy_weights_ci_dict = dict(zip(fl_just_names, fl_accuracy_weights_ci))
fl_accuracy_weights_c_dict = dict(zip(fl_just_names, fl_accuracy_weights_c))
fl_accuracy_weights_i_dict = dict(zip(fl_just_names, fl_accuracy_weights_i))
di_accuracy_weights_ci_dict = dict(zip(di_just_names, di_accuracy_weights_ci))
di_accuracy_weights_c_dict = dict(zip(di_just_names, di_accuracy_weights_c))
di_accuracy_weights_i_dict = dict(zip(di_just_names, di_accuracy_weights_i))

In [ ]:
# Save the accuracy weights
with open('weights_2/accuracy/fl_accuracy_weights_ci.json', 'w') as f:
    json.dump(fl_accuracy_weights_ci_dict, f)

with open('weights_2/accuracy/fl_accuracy_weights_c.json', 'w') as f:
    json.dump(fl_accuracy_weights_c_dict, f)

with open('weights_2/accuracy/fl_accuracy_weights_i.json', 'w') as f:
    json.dump(fl_accuracy_weights_i_dict, f)

with open('weights_2/accuracy/di_accuracy_weights_ci.json', 'w') as f:
    json.dump(di_accuracy_weights_ci_dict, f)

with open('weights_2/accuracy/di_accuracy_weights_c.json', 'w') as f:
    json.dump(di_accuracy_weights_c_dict, f)

with open('weights_2/accuracy/di_accuracy_weights_i.json', 'w') as f:
    json.dump(di_accuracy_weights_i_dict, f)

# Load the accuracy weights
with open('weights_2/accuracy/fl_accuracy_weights_ci.json', 'r') as f:
    fl_accuracy_weights_ci_dict = json.load(f)

with open('weights_2/accuracy/fl_accuracy_weights_c.json', 'r') as f:
    fl_accuracy_weights_c_dict = json.load(f)

with open('weights_2/accuracy/fl_accuracy_weights_i.json', 'r') as f:
    fl_accuracy_weights_i_dict = json.load(f)

with open('weights_2/accuracy/di_accuracy_weights_ci.json', 'r') as f:
    di_accuracy_weights_ci_dict = json.load(f)

with open('weights_2/accuracy/di_accuracy_weights_c.json', 'r') as f:
    di_accuracy_weights_c_dict = json.load(f)

with open('weights_2/accuracy/di_accuracy_weights_i.json', 'r') as f:
    di_accuracy_weights_i_dict = json.load(f)

### Shapley weights

In [ ]:
fl_shapley_weights_borda_ci = wf_fl_ci.shapley_pytorch(method = 'borda')
# fl_shapley_weights_borda_c = wf_fl_c.shapley_pytorch(method = 'borda')
# fl_shapley_weights_borda_i = wf_fl_i.shapley_pytorch(method = 'borda')

di_shapley_weights_borda_ci = wf_di_ci.shapley_pytorch(method = 'borda')
# di_shapley_weights_borda_c = wf_di_c.shapley_pytorch(method = 'borda')
# di_shapley_weights_borda_i = wf_di_i.shapley_pytorch(method = 'borda')

# fl_shapley_weights_plurality_ci = wf_fl_ci.shapley_pytorch(method = 'plurality')
# # fl_shapley_weights_plurality_c = wf_fl_c.shapley_pytorch(method = 'plurality')
# # fl_shapley_weights_plurality_i = wf_fl_i.shapley_pytorch(method = 'plurality')

# di_shapley_weights_plurality_ci = wf_di_ci.shapley_pytorch(method = 'plurality')
# di_shapley_weights_plurality_c = wf_di_c.shapley_pytorch(method = 'plurality')
# di_shapley_weights_plurality_i = wf_di_i.shapley_pytorch(method = 'plurality')

Calculating Shapley values for 16 models


  0%|          | 0/17 [00:00<?, ?it/s]/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
100%|██████████| 16/16 [00:00<00:00, 41.68it/s]


Calculating Shapley values for 16 models


  0%|          | 0/17 [00:00<?, ?it/s]/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/pratik/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
100%|██████████| 16/16 [00:00<00:00, 39.48it/s]


In [ ]:
fl_shapley_weights_borda_ci_dict = dict(zip(fl_just_names, fl_shapley_weights_borda_ci))
# fl_shapley_weights_borda_c_dict = dict(zip(fl_just_names, fl_shapley_weights_borda_c))
# fl_shapley_weights_borda_i_dict = dict(zip(fl_just_names, fl_shapley_weights_borda_i))
di_shapley_weights_borda_ci_dict = dict(zip(di_just_names, di_shapley_weights_borda_ci))
# di_shapley_weights_borda_c_dict = dict(zip(di_just_names, di_shapley_weights_borda_c))
# di_shapley_weights_borda_i_dict = dict(zip(di_just_names, di_shapley_weights_borda_i))

# fl_shapley_weights_plurality_ci_dict = dict(zip(fl_just_names, fl_shapley_weights_plurality_ci))
# # fl_shapley_weights_plurality_c_dict = dict(zip(fl_just_names, fl_shapley_weights_plurality_c))
# # fl_shapley_weights_plurality_i_dict = dict(zip(fl_just_names, fl_shapley_weights_plurality_i))
# di_shapley_weights_plurality_ci_dict = dict(zip(di_just_names, di_shapley_weights_plurality_ci))
# di_shapley_weights_plurality_c_dict = dict(zip(di_just_names, di_shapley_weights_plurality_c))
# di_shapley_weights_plurality_i_dict = dict(zip(di_just_names, di_shapley_weights_plurality_i))

In [ ]:
fl_shapley_weights_borda_ci_dict

{'vgg_cifar_fl_1.pt': 0.0614745089786758,
 'cifar10_net_imagenet_fl_2.pt': 0.024981681004180713,
 'vgg_imagenet_fl_3.pt': -0.000702608564275225,
 'cifar10_net_cifar_fl_3.pt': 0.007790011747511169,
 'cifar10_net_imagenet_fl_3.pt': 0.010879832667332265,
 'vgg_imagenet_fl_0.pt': 0.04769879845463185,
 'vgg_cifar_fl_2.pt': 0.04459245085161717,
 'cifar10_net_cifar_fl_1.pt': 0.018612813636979964,
 'cifar10_net_imagenet_fl_0.pt': 0.0473282650065984,
 'vgg_cifar_fl_3.pt': 0.02400564460231155,
 'cifar10_net_cifar_fl_2.pt': 0.015703290320789827,
 'cifar10_net_imagenet_fl_1.pt': 0.03425018308851678,
 'vgg_cifar_fl_0.pt': 0.10451886086136009,
 'vgg_imagenet_fl_2.pt': 0.012513210986544603,
 'cifar10_net_cifar_fl_0.pt': 0.014331386113886221,
 'vgg_imagenet_fl_1.pt': 0.029943892465559462}

In [ ]:
# Save the weights
with open('weights_2/shapley/fl_shapley_weights_borda_ci.json', 'w') as f:
    json.dump(fl_shapley_weights_borda_ci_dict, f)

# with open('weights_2/shapley/fl_shapley_weights_borda_c.json', 'w') as f:
#     json.dump(fl_shapley_weights_borda_c_dict, f)

# with open('weights_2/shapley/fl_shapley_weights_borda_i.json', 'w') as f:
#     json.dump(fl_shapley_weights_borda_i_dict, f)

with open('weights_2/shapley/di_shapley_weights_borda_ci.json', 'w') as f:
    json.dump(di_shapley_weights_borda_ci_dict, f)

# with open('weights_2/shapley/di_shapley_weights_borda_c.json', 'w') as f:
#     json.dump(di_shapley_weights_borda_c_dict, f)

# with open('weights_2/shapley/di_shapley_weights_borda_i.json', 'w') as f:
#     json.dump(di_shapley_weights_borda_i_dict, f)

# with open('weights_2/shapley/fl_shapley_weights_plurality_ci.json', 'w') as f:
#     json.dump(fl_shapley_weights_plurality_ci_dict, f)

# # with open('weights_2/shapley/fl_shapley_weights_plurality_c.json', 'w') as f:
# #     json.dump(fl_shapley_weights_plurality_c_dict, f)

# # with open('weights_2/shapley/fl_shapley_weights_plurality_i.json', 'w') as f:
# #     json.dump(fl_shapley_weights_plurality_i_dict, f)

# with open('weights_2/shapley/di_shapley_weights_plurality_ci.json', 'w') as f:
#     json.dump(di_shapley_weights_plurality_ci_dict, f)

# with open('weights_2/shapley/di_shapley_weights_plurality_c.json', 'w') as f:
#     json.dump(di_shapley_weights_plurality_c_dict, f)

# with open('weights_2/shapley/di_shapley_weights_plurality_i.json', 'w') as f:
#     json.dump(di_shapley_weights_plurality_i_dict, f)

# Load the weights
# with open('weights_2/shapley/fl_shapley_weights_borda_ci.json', 'r') as f:
#     fl_shapley_weights_borda_ci_dict = json.load(f)

# with open('weights_2/shapley/fl_shapley_weights_borda_c.json', 'r') as f:
#     fl_shapley_weights_borda_c_dict = json.load(f)

# with open('weights_2/shapley/fl_shapley_weights_borda_i.json', 'r') as f:
#     fl_shapley_weights_borda_i_dict = json.load(f)

# with open('weights_2/shapley/di_shapley_weights_borda_ci.json', 'r') as f:
#     di_shapley_weights_borda_ci_dict = json.load(f)

# with open('weights_2/shapley/di_shapley_weights_borda_c.json', 'r') as f:
#     di_shapley_weights_borda_c_dict = json.load(f)

# with open('weights_2/shapley/di_shapley_weights_borda_i.json', 'r') as f:
#     di_shapley_weights_borda_i_dict = json.load(f)

# with open('weights_2/shapley/fl_shapley_weights_plurality_ci.json', 'r') as f:
#     fl_shapley_weights_plurality_ci_dict = json.load(f)

# with open('weights_2/shapley/fl_shapley_weights_plurality_c.json', 'r') as f:
#     fl_shapley_weights_plurality_c_dict = json.load(f)

# with open('weights_2/shapley/fl_shapley_weights_plurality_i.json', 'r') as f:
#     fl_shapley_weights_plurality_i_dict = json.load(f)

# with open('weights_2/shapley/di_shapley_weights_plurality_ci.json', 'r') as f:
#     di_shapley_weights_plurality_ci_dict = json.load(f)

# with open('weights_2/shapley/di_shapley_weights_plurality_c.json', 'r') as f:
#     di_shapley_weights_plurality_c_dict = json.load(f)

# with open('weights_2/shapley/di_shapley_weights_plurality_i.json', 'r') as f:
#     di_shapley_weights_plurality_i_dict = json.load(f)

### Regression weights

In [ ]:
fl_regression_weights_ci = wf_fl_ci.regression_pytorch()
# fl_regression_weights_c = wf_fl_c.regression_pytorch()
# fl_regression_weights_i = wf_fl_i.regression_pytorch()

di_regression_weights_ci = wf_di_ci.regression_pytorch()
# di_regression_weights_c = wf_di_c.regression_pytorch()
# di_regression_weights_i = wf_di_i.regression_pytorch()

Calculating Regression weights for 16 models
Calculating Regression weights for 16 models


In [ ]:
fl_regression_weights_ci_dict = dict(zip(fl_just_names, fl_regression_weights_ci))
# fl_regression_weights_c_dict = dict(zip(fl_just_names, fl_regression_weights_c))
# fl_regression_weights_i_dict = dict(zip(fl_just_names, fl_regression_weights_i))
di_regression_weights_ci_dict = dict(zip(di_just_names, di_regression_weights_ci))
# di_regression_weights_c_dict = dict(zip(di_just_names, di_regression_weights_c))
# di_regression_weights_i_dict = dict(zip(di_just_names, di_regression_weights_i))

In [ ]:
# Save the weights
with open('weights_2/regression/fl_regression_weights_ci.json', 'w') as f:
    json.dump(fl_regression_weights_ci_dict, f)

# with open('weights_2/regression/fl_regression_weights_c.json', 'w') as f:
#     json.dump(fl_regression_weights_c_dict, f)

# with open('weights_2/regression/fl_regression_weights_i.json', 'w') as f:
#     json.dump(fl_regression_weights_i_dict, f)

with open('weights_2/regression/di_regression_weights_ci.json', 'w') as f:
    json.dump(di_regression_weights_ci_dict, f)

# with open('weights_2/regression/di_regression_weights_c.json', 'w') as f:
#     json.dump(di_regression_weights_c_dict, f)

# with open('weights_2/regression/di_regression_weights_i.json', 'w') as f:
#     json.dump(di_regression_weights_i_dict, f)

# Load the weights
# with open('weights_2/regression/fl_regression_weights_ci.json', 'r') as f:
#     fl_regression_weights_ci_dict = json.load(f)

# with open('weights_2/regression/fl_regression_weights_c.json', 'r') as f:
#     fl_regression_weights_c_dict = json.load(f)

# with open('weights_2/regression/fl_regression_weights_i.json', 'r') as f:
#     fl_regression_weights_i_dict = json.load(f)

# with open('weights_2/regression/di_regression_weights_ci.json', 'r') as f:
#     di_regression_weights_ci_dict = json.load(f)

# with open('weights_2/regression/di_regression_weights_c.json', 'r') as f:
#     di_regression_weights_c_dict = json.load(f)

# with open('weights_2/regression/di_regression_weights_i.json', 'r') as f:
#     di_regression_weights_i_dict = json.load(f)